In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
np.random.seed(1337)  # for reproducibility

from collections import Counter

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Dropout
from keras.layers import LSTM

# Input data files are available in the "../input/" directory.

In [ ]:
print('reading CSV')

X, Y, Xt, Yt = [], [], [], []
IDF_dict = Counter()
it = 0
csv = pd.read_csv('../input/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1", header=None)

def process_row(row, x, y):
    sent = row[5]
    x_part = sent.lower().split(' ')
    #if len(x_part) < 12:
    #    x_part = x_part + x_part
    y_part = row[0]
    for tok in x_part:
        IDF_dict[tok] += 1
    x.append(x_part)
    if y_part == 0:
        yy = np.array([0])
    elif y_part == 4:
        yy = np.array([1])
    else:
        raise Exception('Invalid y_part value=' + y_part)
    y.append(yy)


for index, row in csv.iterrows():
    if not it%21 == 0: # TRAINING
        process_row(row, X, Y)

    else:               # TEST
        process_row(row, Xt, Yt)

    it += 1

del csv

In [ ]:
print('build words frequency map')

max_features = 50000
words_map = {}
for index, (word, freq) in enumerate(IDF_dict.most_common(max_features)):
    words_map[word] = index + 1
words_map[''] = 0 # to be 100% sure that empty string encoded as 0

In [ ]:
print('encode input sequences')

def map_words(x):
    maxlen = 0
    for words in x:
        for index, word in enumerate(words):
            if word in words_map:
                words[index] = words_map[word]
            else:
                words[index] = 0
        words.append(0)
        maxlen = max(maxlen, len(words))
    return maxlen
        

maxlen = map_words(X)
maxlen = max(maxlen, map_words(Xt))
X = sequence.pad_sequences(X, padding='post', truncating='post', value=0, maxlen=maxlen)
Xt = sequence.pad_sequences(Xt, padding='post', truncating='post', value=0, maxlen=maxlen)

In [ ]:
print('build model')

batch_size = 32

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

print('Train...')
for index in range(len(X)):
    X[index] = np.array(X[index])
for index in range(len(Xt)):
    Xt[index] = np.array(Xt[index])
X = np.array(X)
Xt = np.array(Xt)
Y = np.array(Y)
Yt = np.array(Yt)

model.fit(X, Y, batch_size=batch_size, epochs=1,validation_data=(Xt, Yt))




score, acc = model.evaluate(Xt, Yt,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)